# Welche Entwickler haben die meisten Commits?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [6]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [7]:
import pandas as pd

query ="MATCH (a:Author)-[:COMMITTED]->(c:Commit)-[:CONTAINS_CHANGE]->(:Change)-[:MODIFIES]->(file:File) WHERE NOT c:Merge RETURN a.name as developer, count(distinct c) as commits"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [8]:
df.head()

commits      developer
0        1  Joseph Walton
1       23       Noel Yap
2        2  Robin Stevens
3        1   Chris Barrow
4        1    Arjen Stolk

Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der der Anzahl der Commits (commits) sortiert und die ersten zehn Entwickler (developer) extrahiert. Die Werte werden in den Variablen <font face="Courier">x_commits</font> und <font face="Courier">y_developer</font> gespeichert.

In [9]:
import numpy as np

a=df.sort_values('commits',ascending=False)
b=a.head(10)

def x_commits(x):
    x=b['commits'].tolist()
    return x


def y_developer(y):
    y=b['developer'].tolist()
    return(y)



print(x_commits(b))
print(y_developer(b))
print(b)

[314, 257, 132, 121, 100, 92, 87, 53, 37, 36]
['David Saff', 'dsaff', 'Stefan Birkner', 'kbeck', 'Tibor Digana', 'Kevin Cooney', 'Marc Philipp', 'egamma', 'emeade', 'pimterry']
     commits       developer
121      314      David Saff
128      257           dsaff
171      132  Stefan Birkner
23       121           kbeck
131      100    Tibor Digana
127       92    Kevin Cooney
94        87    Marc Philipp
29        53          egamma
125       37          emeade
110       36        pimterry


# 4. Visualisierung
Die Daten werden mittels eines Bar Charts visualisiert (https://plot.ly/python/horizontal-bar-charts/, x=x_commits, y=y_developer).

In [10]:
import plotly.plotly as py
import plotly.graph_objs as go

data = [go.Bar(
            x=x_commits(b),
            y=y_developer(b),
            orientation = 'h'
)]

py.iplot(data, filename='Entwickler mit den meisten Commits')



In [12]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [14]:
import pygal 
line_chart = pygal.HorizontalBar()
line_chart.title = 'Entwickler mit den meisten Commits'
for einzelnerEntwickler in y_developer(b):
    index= y_developer(b).index(einzelnerEntwickler)
    #dazugehörige frequency herausbekommen
    anzahl=x_commits(b)[index]
    line_chart.add(einzelnerEntwickler, anzahl)
display(HTML(base_html.format(rendered_chart=line_chart.render(is_unicode=True))))